A tiny RAG QA demo
- LLM: gpt-3.5-turbo
- external data: NCC 2022 (National Construction Code)
- Vector store: ChromaDB

In [4]:
%%cmd
pip install --upgrade pip
pip install -r requirements.txt 

Microsoft Windows [Version 10.0.19045.3324]
(c) Microsoft Corporation. All rights reserved.

(venv310) d:\Documents\work\docqa>pip install --upgrade pip
  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\Documents\work\ask_api\ask-api\venv310\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>pip install -r requirements.txt 
     ---------------------------------------- 2.1/2.1 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.3.0
    Uninstalling pydantic-2.3.0:
      Successfully uninstalled pydantic-2.3.0



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>

In [22]:
%jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


check virtual environment

In [1]:
import sys
sys.prefix

'd:\\Documents\\work\\vscode_workspace\\docqa\\.venv'

In [1]:
import os
import glob
import pprint
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import openai
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from tqdm.notebook import tqdm




openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import langchain
langchain.__version__

'0.0.157'

# 1. Without Langchain

## 1.1 Injest NCC 2022 documents to ChromaDB

Create collection of NCC 2022 data in chromadb.

In [5]:
import chromadb
chromadb_client = chromadb.Client(Settings(
    chroma_db_impl='duckdb+parquet', #chroma_db_impl="duckdb",
    persist_directory="./chroma_db_ncc/"
))
# chromadb_client = chromadb.PersistentClient(path = "./chroma_db")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022 = chromadb_client.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 

# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 api_key=os.environ['OPENAI_API_KEY'],
#                 model_name="text-embedding-ada-002"
#             )

# collection_ncc_openai = client.create_collection(name="ncc2022openai", embedding_function=openai_ef) 

Using embedded DuckDB with persistence: data will be stored in: ./chroma_db_ncc/


In [6]:
chromadb_client.list_collections()

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]

In [22]:
#chromadb_client.delete_collection(name="ncc2022")

In [7]:
file_folder = "D:/Documents/data/national construction code/2022/*.pdf"
for f in glob.glob(file_folder):
    print(f'processing {f}')
    pdf_loader = PyPDFLoader(f)
    docs = pdf_loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(docs)
    collection_ncc2022.add(
        documents=[d.page_content for d in docs],
        metadatas=[d.metadata for d in docs],
        ids=[d.metadata['source'] + str(d.metadata['page']) for d in docs]
    )
chromadb_client.persist()

processing D:/Documents/data/national construction code/2022\abcb-housing-provisions-2022-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-combined-vol2-housing-provisions-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-consolidated-performance-requirements-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-one-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-three-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-two-20230501.pdf


 impossible to decode XFormObject /Fm0


True

## 1.2 Test QA

Load collection of NCC 2022 data.

In [8]:
chromadb_client_2 = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma_db_ncc/",

))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022 = chromadb_client_2.get_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 


Using embedded DuckDB with persistence: data will be stored in: ./chroma_db_ncc/


In [9]:
chromadb_client_2.list_collections()

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]

In [10]:
collection_ncc2022 = chromadb_client_2.get_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 
collection_ncc2022.count()

3239

In [12]:
def build_prompt_with_context(question, context):
    system_prompt = {
        "role": "system",
        "content": """You are a helpful assistant who knows a lot about architecture standards in Australia. 
        I will ask you questions about architecture standards. I will provide a number of relevant paragraphs extracted from 
        the architecture standards for your reference."""
    }
    user_prompt = {
        "role": "user",
        "content": f"""
        Please help me on the question delimited by ``` regarding archituecture standards in Australia. The texts delimited by === are 
        a number of paragraphs extracted from National Construction Code. You can use the texts as reference. Please provide your reply in format of json with 3 fields: "answer",
        "confidence score" between 0 to 1, and "source" which are the original clauses supporting your answer. 
        
        Question:```
        {question}
        ```
        Texts:===
        {os.linesep.join(context)}
        ===
        """
    }
    return [system_prompt,
            user_prompt]





<b> Ask Question in different ways: </b> 

 The question is "What's the maximum travel distance between two fire exits in a shopping center?".

 Ask it in different ways.

In [39]:
question = "What class of building does a shopping center belong to? What travel distance should two fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
# response

In [40]:
# check retrieval
texts

{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf190',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf248',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf245']],
 'embeddings': None,
 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 191required  by (4) is to be of a size that accommodates all the occupants from the fire compartment  being evacuated.  \n(6) In a Class 9b early childhood centre , the clear area required  by (4) must accommodate all occupants of the early  \nchildhood centre . \n(7) The clear area required  by (4) must be connected to the horizontal exit  by an unobstructed path that has at least the  \ndimensions required  for the horizontal exit  and may include the area of the unobstructed path.  \n(8) Each fire compartment  required  by C3D6(2)  must be served by not less than 2 horizontal exits , each locat

In [41]:
# check augmented prompt
print(messages[1]['content'])


        Please help me on the question delimited by ``` regarding archituecture standards in Australia. The texts delimited by === are 
        a number of paragraphs extracted from National Construction Code. You can use the texts as reference. Please provide your reply in format of json with 3 fields: "answer",
        "confidence score" between 0 to 1, and "source" which are the original clauses supporting your answer. 
        
        Question:```
        What class of building does a shopping center belong to? What travel distance should two fire staires of shopping center be?
        ```
        Texts:===
        Access and egress
NCC 2022 Volume One - Building Code of Australia Page 191required  by (4) is to be of a size that accommodates all the occupants from the fire compartment  being evacuated.  
(6) In a Class 9b early childhood centre , the clear area required  by (4) must accommodate all occupants of the early  
childhood centre . 
(7) The clear area required  by (4) m

In [42]:
# check answer
result = response['choices'][0]['message']['content']
print(result)

{
    "answer": {
        "Class of building": "Shopping centers usually belong to Class 6 buildings in Australia standards.",
        "Fire stairs travel distance": "The two fire stairs within a shopping center should be a minimum of 9 meters apart from each other.",
    },
    "confidence score": 0.85,
    "source": "NCC 2022 Volume One - Building Code of Australia Page 191"
}


Ask in another way.

In [45]:
question = "What travel distance should two fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
# response

# check retrieval
# texts

# check augmented prompt
# print(messages[1]['content'])

# check answer
result = response['choices'][0]['message']['content']
print(result)

{
    "answer": "The travel distance between two fire staircases of a shopping center should be measured along the path of travel through the point at which travel in different directions to those exits is available, as determined in accordance with the standards.",
    "confidence score": 0.85,
    "source": "NCC 2022 Volume One - Building Code of Australia Page 194"
}


Ask again.

In [36]:
question = "What class of building does a shopping center belong to?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
# response


# check retrieval
# texts

# check augmented prompt
# print(messages[1]['content'])

# check answer
result = response['choices'][0]['message']['content']
print(result)

{
    "answer": "A shopping center belongs to Class 6 building.",
    "confidence score": 1,
    "source": "NCC 2022 Volume One - Building Code of Australia Page 633, Part A6"
}


In [38]:
question = "What's the maximum travel distance between two fire exits in a shopping center?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
# response


# check retrieval
# texts

# check augmented prompt
# print(messages[1]['content'])

# check answer
result = response['choices'][0]['message']['content']
print(result)

{
    "answer": "The maximum travel distance between two fire exits in a shopping center must be not more than 60 meters.",
    "confidence score": 1,
    "source": "NCC 2022 Volume One - Building Code of Australia Page 186"
}


# 2. Use Langchain 

### 2.1 Create langchian vectorstore backed by chroma db from documents

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

raw_documents = PyPDFDirectoryLoader("D:/Documents/data/national construction code/2022/").load()

print(f"raw_documents {len(raw_documents)}")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500,
                                                                     chunk_overlap=20,
                                                                    )
splitted_documents = text_splitter.split_documents(raw_documents)

print(f"splitted docuemnts {len(splitted_documents)}")



In [9]:
from tqdm import tqdm
chroma_db = Chroma(
    collection_name="building_code",
    persist_directory="./chroma_db_2",
    embedding_function=SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

with tqdm(total=len(splitted_documents)) as pbar:
    for d in splitted_documents:
        chroma_db.add_documents([d])
        pbar.update(1)
chroma_db.persist()

Using embedded DuckDB with persistence: data will be stored in: ./chroma_db_2
100%|██████████| 5319/5319 [16:02<00:00,  5.53it/s]


### 2.2 Load existing langchain chroma db

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
chroma_db = Chroma(
    collection_name="building_code",
    persist_directory="./chroma_db_2",
    embedding_function=SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    )

Using embedded DuckDB with persistence: data will be stored in: ./chroma_db_2


In [12]:
# validate retrieval 
question = "What's the maximum travel distance between two fire exits in a shopping center?"


texts = chroma_db.max_marginal_relevance_search(query=question, n_results=3)
texts

[Document(page_content='The maximum distance of travel, as required  by D2D5(1)(a)(i) , may be increased from 6 m to 12 m.  (c)\nThe maximum distance of travel from a single exit serving the storey  at the level of egress to a road or open  (d)\nspace , as required  by D2D5(1)(a)(ii) , may be increased from 20 m to 30 m.  \nThe maximum distance between alternative exits, as required  by D2D6(c)(i) , may be increased from 45 m to 60  (e)\nm.S18C4\n (1 May 2023)', metadata={'source': 'D:\\Documents\\data\\national construction code\\2022\\ncc2022-volume-one-20230501.pdf', 'page': 270}),
 Document(page_content='the fire safety system  installed in the building; and  (b)\nthe function or use of the building; and  (c)\nthe number of storeys  passed through by the exits; and  (d)\nfire brigade  intervention.  (e)\n\u200aD1P6\u200a \xa0Paths of travel to exits  \n[2019: DP6]  \nSo that occupants can safely evacuate the building, paths of travel to exits must have dimensions appropriate to—  \

In [16]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)


# Other inputs
question = "What's the maximum travel distance between two fire exits in a shopping center?"

In [ ]:
response = llm_chain.run(question)
response

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Run
retriever = MultiQueryRetriever(
    retriever=chroma_db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query=question
)

In [17]:
unique_docs

[]

In [24]:
chroma_retriever = chroma_db.as_retriever()
chroma_retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x000001E207D6B160>, search_type='similarity', search_kwargs={})

In [27]:
question

"What's the maximum travel distance between two fire exits in a shopping center?"

In [25]:
chroma_retriever.vectorstore._collection.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )

{'ids': [['e1af34f1-b783-11ef-9f30-f875a43ff3d5',
   'e1730c69-b783-11ef-9740-f875a43ff3d5',
   'cca1f0bb-b783-11ef-8b02-f875a43ff3d5']],
 'embeddings': None,
 'documents': [['The maximum distance of travel, as required  by D2D5(1)(a)(i) , may be increased from 6 m to 12 m.  (c)\nThe maximum distance of travel from a single exit serving the storey  at the level of egress to a road or open  (d)\nspace , as required  by D2D5(1)(a)(ii) , may be increased from 20 m to 30 m.  \nThe maximum distance between alternative exits, as required  by D2D6(c)(i) , may be increased from 45 m to 60  (e)\nm.S18C4\n (1 May 2023)',
   'reduced to –/60/15; and  \nnon-loadbearing  fire-resisting  lift and stair shafts , as required  by Specification 5 , may be reduced to –/60/60.  (ii)\nThe maximum distance of travel, as required  by D2D5(1)(a)(i) , may be increased from 6 m to 12 m.  (c)\nThe maximum distance of travel from a single exit serving the storey  at the level of egress to a road or open  (d)\nspa

In [26]:
chroma_retriever.vectorstore._collection.peek()

{'ids': ['9ed05d98-b783-11ef-bf7d-f875a43ff3d5',
  '3e2b0dd3-b784-11ef-ae5c-f875a43ff3d5',
  '12d95b2d-b783-11ef-ad2e-f875a43ff3d5',
  '12fbda36-b783-11ef-a1ae-f875a43ff3d5',
  '92228ce8-b782-11ef-81bd-f875a43ff3d5',
  '922b7b1b-b782-11ef-9c15-f875a43ff3d5',
  '9242356f-b782-11ef-8cbe-f875a43ff3d5',
  '925bb288-b782-11ef-b368-f875a43ff3d5',
  '927975c2-b782-11ef-8d2b-f875a43ff3d5',
  '9294c793-b782-11ef-8c0e-f875a43ff3d5'],
 'embeddings': [[-0.032434746623039246,
   0.02366737276315689,
   0.009624062106013298,
   -0.07014305144548416,
   -0.04840610548853874,
   0.04538629576563835,
   0.0686650201678276,
   -0.047600291669368744,
   -0.004025368485599756,
   0.006464432924985886,
   0.03895602002739906,
   -0.10184532403945923,
   0.022964252158999443,
   0.06617149710655212,
   -0.039269763976335526,
   -0.020897123962640762,
   -0.012863482348620892,
   0.016205260530114174,
   -0.03754435479640961,
   -0.028949890285730362,
   0.02972286194562912,
   -0.05922990292310715,
   -0.01